In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [22]:
STORM_NUMBER = 14
TITLE = "25-26 January 2021"
SUBTITLE = "8 PM 26 January 2021"
sts = utc(2021, 1, 26, 0)
ets = utc(2021, 12, 30, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [6]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [23]:
cull = ['CLUI4', 'IA-ST-3', 'AMOI4', 'IA-LN-30', 'TLDI4', 'VNTI4', 623, 'GRWI4', 'MSHI4', 'ZRGI4', 360, 380]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
    ['IA-DV-3', 4],
    [258, 5],
    ['IA-SR-19', 8],
    [305, 8],
    [365, 8],
    ['PEAI4', 8],
    ['HRFI4', 8],
    [29, 11.5],
    ['IA-DL-33', 10],
    ['MHRI4', 7],
    [528, 7],
    ['IA-JH-1', 7],
    
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=4, upper=6)
res = mp.postprocess(filename='210126.png')
mp.close()

     state  wfo  val       lon      lat                               geo  \
258     IA  DMX  5.0  -94.8800  42.2500    POINT (421199.998 -292600.224)   
450     07  MKX  5.9  -87.9200  43.0300    POINT (977587.280 -146530.626)   
451     IL  LOT  5.9  -87.8200  42.0800   POINT (1000947.121 -250103.106)   
452     WI  MKX  5.9  -88.4300  42.5900    POINT (943442.466 -201085.075)   
453     WI  MKX  5.8  -90.0700  43.1800    POINT (802794.049 -153443.681)   
...    ...  ...  ...       ...      ...                               ...   
1902    WI  MKX  4.0  -87.8117  43.6567     POINT (976034.361 -76111.577)   
1903    NV  VEF  4.0 -116.0033  36.2786  POINT (-1428016.472 -832323.901)   
1905    IA  DMX  4.0  -94.6606  42.7292    POINT (435824.046 -238257.441)   
1906    CO  PUB  4.0 -105.1472  38.7731   POINT (-446524.115 -678252.838)   
1907    IL  LOT  4.0  -88.1639  42.1153    POINT (972383.689 -250355.212)   

      used_for_analysis  nwsli  plotme source  
258                True    